In [4]:
import numpy as np
from utils.download_from_GCP import download_table_to_local_as_one_file
import pandas as pd
import os
from sklearn.metrics import ndcg_score, roc_auc_score
from tqdm import tqdm
from multiprocessing import Pool
import multiprocessing as mp
import pathlib
import xlearn as xl

In [5]:
LANGS = [
    "Hindi",
    # "Tamil",
#     "Telugu",
#     "Kannada",
#     "Punjabi",
#     "Odia",
#     "Bengali",
#     "Marathi",
#     "Malayalam",
#     "Gujarati",
]
# DAYS_OF_DATA_CONSIDERED = 7
rating_def_dict = {
        "vplay": "is_vp_succ",
        "like": "is_like",
# #         "share": "is_share",
# #         "fav": "is_fav",
#         "vplay_skip": "is_vp_skip",
#         "vplay2": "is_vp_succ2",
#     "vclick": "is_vp_click"
}

# table_path = "maximal-furnace-783.rohitrr.test_temp_q1_table_Malayalam_is_vp_succ2"
# local_save_path = "./train_test_data_models/Malayalam/is_vp_succ2"
out_file_name = "test_q1.csv"
min_pos_labels = 10
min_total_user_events = 20
num_users_to_consider = 20000
RANDOM_SEED = 9745
TEST_DATA_FILE_NAME = "test"

In [6]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

In [7]:
def predict_scores(test_file_path, test_model_path, predicted_output_folder_path, 
                   rating_def, lang, predicted_output_file_name = "predicted_output.txt"):
    pathlib.Path(predicted_output_folder_path).mkdir(parents = True, exist_ok = True)
    predicted_output_file_path = os.path.join(predicted_output_folder_path,
                                             predicted_output_file_name)
    res_out_path = os.path.join(predicted_output_folder_path, "results.csv")
    
    print(f"Predicting labels for {rating_def}-{lang} .... ")
    ffm_model = xl.create_ffm() 
    ffm_model.setTest(test_file_path)
    ffm_model.setSigmoid()
    ffm_model.predict(test_model_path, predicted_output_file_path)
    print(f"Predicted labels stored in {predicted_output_file_path}")
    
def calculate_user_level_metric_scores(true_vals, predicted_vals):
        user_at_5_ndcg_score = ndcg_score(true_vals[np.newaxis, :], predicted_vals[np.newaxis, :],
                                         k = 5)
        user_at_10_ndcg_score = ndcg_score(true_vals[np.newaxis, :], predicted_vals[np.newaxis, :],
                                         k = 10)
        user_auc_score = roc_auc_score(true_vals, predicted_vals)
        relevant_recommendations = np.where(true_vals == 1)[0]
        ordered_recommendations = np.argsort(-predicted_vals)
        user_mapk_at_5_score = apk(relevant_recommendations, ordered_recommendations, k=5)
        user_mapk_at_10_score = apk(relevant_recommendations, ordered_recommendations, k=10)
        return [user_at_5_ndcg_score, user_at_10_ndcg_score, 
                user_auc_score, user_mapk_at_5_score, user_mapk_at_10_score]
    #     print("appended")
    
def get_results(test_file_path, predicted_results_path,
                lang, rating_def):
    print("Getting results for {}-{}".format(lang, rating_def))
    print(f"Opening {predicted_results_path} and {test_file_path} - reading input ............")
    with open(predicted_results_path) as f:
        lines = f.readlines()
        predicted_scores = [float(score.replace('\n','')) for score in lines]
    user_mappings = []
    scores = []
    with open(test_file_path) as f:
        for l in f:
            user_mappings.append(int(l.split(':')[1]))
            scores.append(int(l[0]))
    dataframe_dict = {
        "user_mapping": user_mappings,
        "score": scores,
        "predicted_score": predicted_scores
    }
    print(f"Calculating overall AUC scores - {lang}-{rating_def}")
    auc_overall = roc_auc_score(scores, predicted_scores)
    print(f"Overall AUC scores computed - {lang}-{rating_def}")
    
    df = pd.DataFrame(dataframe_dict)
    print(f"Created test dataframe for {lang}-{rating_def}")
    
    print(f"Separating data into groups {lang}-{rating_def} .....")
    agg_df = df[['user_mapping', 'score']].groupby(['user_mapping']).agg(['sum', 'count'])
    agg_df = agg_df[(agg_df["score"]["sum"] >= min_pos_labels) \
                     & (agg_df["score"]["count"] >= min_total_user_events) \
                   & (agg_df["score"]["sum"] != agg_df["score"]["count"])]
    selected_user_mappings = agg_df.sample(n = num_users_to_consider, replace=False).index.values

    df = df[df.user_mapping.isin(selected_user_mappings)]
    df = df.sort_values("user_mapping")
    all_user_mapping = df.user_mapping.values
    scores = df.score.values
    predicted_scores = df.predicted_score.values
    ukeys, index = np.unique(all_user_mapping, True)
    user_level_true_vals = np.split(scores, index[1:])
    user_level_predicted_vals = np.split(predicted_scores, index[1:])
    input_vals = list(zip(user_level_true_vals, user_level_predicted_vals))
    print(f"Input separated into groups for {lang}-{rating_def}")
    print(f"Computing different scores for {lang}-{rating_def}")
    with Pool(processes = 48) as sub_pool:
        res = sub_pool.starmap(calculate_user_level_metric_scores, input_vals)       
    means = np.array(res).mean(axis = 0)
    
    results_dict = {
        "Rating Definition": rating_def,
        "Language": lang,
        "AUC score - Overall": auc_overall,
        "NDCG@5 Score - User Level": means[0],
        "NDCG@10 Score - User Level": means[1],
        "AUC score - User Level": means[2],
        "MAPK@5 score - User Level": means[3],
        "MAPK@10 score - User Level": means[4]
        }
    print("Completed computing results for {} {}".format(lang, rating_def))
    return results_dict

In [8]:
results = []
for lang in LANGS:
    for rating_key, rating_def in rating_def_dict.items():
        test_file_path = f"./train_test_data_models/{lang}/{rating_def}/{TEST_DATA_FILE_NAME}.txt"
        trained_model_path = f"./train_test_data_models/{lang}/{rating_def}/out/model.out"
        predicted_output_folder_path = f"./train_test_data_models/{lang}/{rating_def}/predicted_results"
        predicted_output_file_name = "predicted_scores.txt"
        predict_scores(test_file_path, trained_model_path, predicted_output_folder_path,
                      lang, rating_def, predicted_output_file_name=predicted_output_file_name)
        predicted_output_file_path = os.path.join(predicted_output_folder_path, 
                                                 predicted_output_file_name)
        temp_res = get_results(test_file_path, predicted_output_file_path, 
                                   lang, rating_def)
        results.append(temp_res)
print(results)

Predicting labels for Hindi-is_vp_succ .... 
Predicted labels stored in ./train_test_data_models/Hindi/is_vp_succ/predicted_results/predicted_scores.txt
Getting results for Hindi-is_vp_succ
Opening ./train_test_data_models/Hindi/is_vp_succ/predicted_results/predicted_scores.txt and ./train_test_data_models/Hindi/is_vp_succ/test.txt - reading input ............
Calculating overall AUC scores - Hindi-is_vp_succ
Overall AUC scores computed - Hindi-is_vp_succ
Created test dataframe for Hindi-is_vp_succ
Separating data into groups Hindi-is_vp_succ .....
Input separated into groups for Hindi-is_vp_succ
Computing different scores for Hindi-is_vp_succ
Completed computing results for Hindi is_vp_succ
Predicting labels for Hindi-is_like .... 
Predicted labels stored in ./train_test_data_models/Hindi/is_like/predicted_results/predicted_scores.txt
Getting results for Hindi-is_like
Opening ./train_test_data_models/Hindi/is_like/predicted_results/predicted_scores.txt and ./train_test_data_models/Hin

In [10]:
# pd.DataFrame(results).to_csv("results/small_lang_metrics.csv",index = False, header=False, mode='a')
pd.DataFrame(results).to_csv("results/hindi_lang_metrics.csv",index = False)

In [56]:
results

[{'Rating Definition': 'is_vp_succ',
  'Language': 'Hindi',
  'NDCG@5 Score - User Level': 0.6191154258349969,
  'NDCG@10 Score - User Level': 0.5788642444685365,
  'AUC score - User Level': 0.7583088322075305,
  'MAPK@5 score - User Level': 0.505635666666687,
  'MAPK@10 score - User Level': 0.42529122619047893},
 {'Rating Definition': 'is_vp_skip',
  'Language': 'Hindi',
  'NDCG@5 Score - User Level': 0.8319679011497992,
  'NDCG@10 Score - User Level': 0.7768347865823756,
  'AUC score - User Level': 0.6957142151777189,
  'MAPK@5 score - User Level': 0.7676521666666438,
  'MAPK@10 score - User Level': 0.6711754305555426},
 {'Rating Definition': 'is_vp_succ2',
  'Language': 'Hindi',
  'NDCG@5 Score - User Level': 0.6152171977024968,
  'NDCG@10 Score - User Level': 0.5749213601676259,
  'AUC score - User Level': 0.7671716924660485,
  'MAPK@5 score - User Level': 0.5028325000000198,
  'MAPK@10 score - User Level': 0.42285327182539734},
 {'Rating Definition': 'is_vp_succ',
  'Language': 'K

### Scrap code

In [6]:
for lang in LANGS:
    for rating_key, rating_def in rating_def_dict.items():
        print("Getting results for {}-{}".format(lang, rating_def))
        predicted_results_path = f"./train_test_data_models/{lang}/{rating_def}/predicted_results/predicted_scores.txt"
        test_file_path = f"./train_test_data_models/{lang}/{rating_def}/test.txt"
        print(f"Opening {predicted_results_path} and {test_file_path} - reading input ............")
        with open(predicted_results_path) as f:
            lines = f.readlines()
            predicted_scores = [float(score.replace('\n','')) for score in lines]
        user_mappings = []
        scores = []
        with open(test_file_path) as f:
            for l in f:
                user_mappings.append(int(l.split(':')[1]))
                scores.append(int(l[0]))
        dataframe_dict = {
            "user_mapping": user_mappings,
            "score": scores,
            "predicted_score": predicted_scores
        }
        df = pd.DataFrame(dataframe_dict)
        print(f"Created test dataframe for {lang}-{rating_def}")

        print(f"Separating data into groups {lang}-{rating_def} .....")
        agg_df = df[['user_mapping', 'score']].groupby(['user_mapping']).agg(['sum', 'count'])
        agg_df = agg_df[(agg_df["score"]["sum"] >= min_pos_labels) \
                         & (agg_df["score"]["count"] >= min_total_user_events) \
                       & (agg_df["score"]["sum"] != agg_df["score"]["count"])]
        selected_user_mappings = agg_df.sample(n = num_users_to_consider, replace=False).index.values

        df = df[df.user_mapping.isin(selected_user_mappings)]
        df = df.sort_values("user_mapping")
        all_user_mapping = df.user_mapping.values
        scores = df.score.values
        predicted_scores = df.predicted_score.values
        ukeys, index = np.unique(all_user_mapping, True)
        user_level_true_vals = np.split(scores, index[1:])
        user_level_predicted_vals = np.split(predicted_scores, index[1:])
        input_vals = list(zip(user_level_true_vals, user_level_predicted_vals))
        print(f"Input separated into groups for {lang}-{rating_def}")
        print(f"Computing scores for {lang}-{rating_def}")
    #     with Pool(processes = 24) as sub_pool:
    #         res = sub_pool.starmap(calculate_metric_scores, input_vals)
        res = []
        for true_vals, predicted_vals in tqdm(zip(user_level_true_vals, user_level_predicted_vals)):
#             print(true_vals, predicted_vals)
            temp_res = calculate_user_level_metric_scores(true_vals, predicted_vals)
            res.append(temp_res)

        means = np.array(res).mean(axis = 0)

        results_dict = {
            "Rating Definition": rating_key,
            "Language": lang,
            "NDCG Score - User Level": means[0],
            "AUC score - User Level": means[1],
            "MAPK score - User Level": means[2]
            }
        print(results_dict)

Getting results for Malayalam-is_vp_skip
Opening ./train_test_data_models/Malayalam/is_vp_skip/predicted_results/predicted_scores.txt and ./train_test_data_models/Malayalam/is_vp_skip/test.txt - reading input ............


KeyboardInterrupt: 

In [9]:
for lang in LANGS:
    for rating_key, rating_def in rating_def_dict.items():
        print("Getting results for {}-{}".format(lang, rating_def))
        predicted_results_path = f"./train_test_data_models/{lang}/{rating_def}/predicted_results/predicted_scores.txt"
        test_file_path = f"./train_test_data_models/{lang}/{rating_def}/test.txt"
        print("Getting results for {}-{}".format(lang, rating_def))
        print(f"Opening {predicted_results_path} and {test_file_path} - reading input ............")
        with open(predicted_results_path) as f:
            lines = f.readlines()
            predicted_scores = [float(score.replace('\n','')) for score in lines]
        user_mappings = []
        scores = []
        with open(test_file_path) as f:
            for l in f:
                user_mappings.append(int(l.split(':')[1]))
                scores.append(int(l[0]))
        dataframe_dict = {
            "user_mapping": user_mappings,
            "score": scores,
            "predicted_score": predicted_scores
        }
        print(f"Calculating overall AUC scores - {lang}-{rating_def}")
        auc_overall = roc_auc_score(scores, predicted_scores)
        print(f"Overall AUC scores computed - {lang}-{rating_def}")

        df = pd.DataFrame(dataframe_dict)
        print(f"Created test dataframe for {lang}-{rating_def}")

        print(f"Separating data into groups {lang}-{rating_def} .....")
        agg_df = df[['user_mapping', 'score']].groupby(['user_mapping']).agg(['sum', 'count'])
        agg_df = agg_df[(agg_df["score"]["sum"] >= min_pos_labels) \
                         & (agg_df["score"]["count"] >= min_total_user_events) \
                       & (agg_df["score"]["sum"] != agg_df["score"]["count"])]
        selected_user_mappings = agg_df.sample(n = num_users_to_consider, replace=False).index.values

        df = df[df.user_mapping.isin(selected_user_mappings)]
        df = df.sort_values("user_mapping")
        all_user_mapping = df.user_mapping.values
        scores = df.score.values
        predicted_scores = df.predicted_score.values
        ukeys, index = np.unique(all_user_mapping, True)
        user_level_true_vals = np.split(scores, index[1:])
        user_level_predicted_vals = np.split(predicted_scores, index[1:])
        input_vals = list(zip(user_level_true_vals, user_level_predicted_vals))
        print(f"Input separated into groups for {lang}-{rating_def}")
        print(f"Computing scores for {lang}-{rating_def}")
        with Pool(processes = 48) as sub_pool:
            res = sub_pool.starmap(calculate_user_level_metric_scores, input_vals)
#             res = []
#             for true_vals, predicted_vals in tqdm(zip(user_level_true_vals, user_level_predicted_vals)):
#         #             print(true_vals, predicted_vals)
#                 temp_res = calculate_user_level_metric_scores(true_vals, predicted_vals)
#                 res.append(temp_res)        
        means = np.array(res).mean(axis = 0)

        results_dict = {
            "Rating Definition": rating_def,
            "Language": lang,
            "AUC score - Overall": auc_overall,
            "NDCG@5 Score - User Level": means[0],
            "NDCG@10 Score - User Level": means[1],
            "AUC score - User Level": means[2],
            "MAPK@5 score - User Level": means[3],
            "MAPK@10 score - User Level": means[4]
            }
        print("Completed computing results for {} {}".format(lang, rating_def))
        print(results_dict)

Getting results for Malayalam-is_vp_skip
Getting results for Malayalam-is_vp_skip
Opening ./train_test_data_models/Malayalam/is_vp_skip/predicted_results/predicted_scores.txt and ./train_test_data_models/Malayalam/is_vp_skip/test.txt - reading input ............
Calculating overall AUC scores - Malayalam-is_vp_skip
Overall AUC scores computed - Malayalam-is_vp_skip
Created test dataframe for Malayalam-is_vp_skip
Separating data into groups Malayalam-is_vp_skip .....
Input separated into groups for Malayalam-is_vp_skip
Computing scores for Malayalam-is_vp_skip
Completed computing results for Malayalam is_vp_skip
Getting results for Malayalam-is_vp_succ2
Getting results for Malayalam-is_vp_succ2
Opening ./train_test_data_models/Malayalam/is_vp_succ2/predicted_results/predicted_scores.txt and ./train_test_data_models/Malayalam/is_vp_succ2/test.txt - reading input ............


FileNotFoundError: [Errno 2] No such file or directory: './train_test_data_models/Malayalam/is_vp_succ2/predicted_results/predicted_scores.txt'

In [10]:
results_dict

{'Rating Definition': 'is_vp_skip',
 'Language': 'Malayalam',
 'AUC score - Overall': 0.7403779265112256,
 'NDCG@5 Score - User Level': 0.7511589330745353,
 'NDCG@10 Score - User Level': 0.714626302675468,
 'AUC score - User Level': 0.6773018945198868,
 'MAPK@5 score - User Level': 0.6617723333333213,
 'MAPK@10 score - User Level': 0.5869704742063461}

In [ ]:
df = pd.DataFrame(results)
df.to_csv("./aggregated_results/other_metrics_1.csv", mode='a', 
          index=False, header=False)

In [48]:
len(predicted_scores), len(scores), len(all_user_mapping)

(62711999, 2447441, 2447441)

In [174]:
%%time
user_mapping_df = df[df.user_mapping == user_mapping]

CPU times: user 47.1 ms, sys: 19.9 ms, total: 67 ms
Wall time: 65.6 ms


In [47]:
len(df.predicted_score.values)

2447441

In [183]:
temp_df = df[df.user_mapping.isin(selected_user_mappings)]

In [132]:
temp_df = df[df.userId.isin(selected_userIds)]

In [49]:
df = df.sort_values("user_mapping")
all_user_mapping = df.user_mapping.values
scores = df.score.values
precicted_scores = df.predicted_score.values
len(all_user_mapping), len(scores), len(predicted_scores)

(2447441, 2447441, 62711999)

In [50]:
df.head(5)

,user_mapping,score,predicted_score
7782684,68939,0,0.788298
7032797,68939,0,0.687772
7017182,68939,1,0.500768
12573382,68939,0,0.313040
53359825,68939,0,0.577004


In [23]:
df.groupby(['postId']).sum().sort_values(by=['score'], ascending=False).head(5)

,userId,tagId,score,predicted_score
postId,,,,
9857235382,119103269175624,199314537408,77330,25836.966834
3365384382,116716063066117,195051641932,72874,25429.166583
9329535382,106995824611773,178190688536,70690,23113.308994
9062058282,98212164199390,163874211312,68897,21243.150008
1372802382,90735147065879,152006896006,68039,19701.435099


In [27]:
agg_df = df[['userId', 'score']].groupby(['userId']).agg(['sum', 'count'])

In [134]:
df.shape

(55914986, 5)

In [31]:
agg_df.columns

MultiIndex([('score',   'sum'),
            ('score', 'count')],
           )

In [32]:
agg_df.index

Int64Index([     53871,      55059,      55062,      55564,      56773,
                 57106,      57358,      57549,      58644,      58785,
            ...
            2777448214, 2777458053, 2777461328, 2777461595, 2777473212,
            2777478170, 2777479686, 2777479756, 2777481430, 2777483046],
           dtype='int64', name='userId', length=1125986)

In [35]:
agg_df[('score', 'sum')].values

array([18,  3, 22, ...,  1,  3,  1])

In [44]:
temp_df = agg_df[(agg_df["score"]["sum"] >= min_pos_labels) & (agg_df["score"]["count"] >= min_total_user_events)]

In [49]:
temp_df["userId"]

KeyError: 'userId'

In [51]:
t = agg_df.sample(n = 20000, replace=False).head(5)

In [54]:
t.index.values

array([ 176951320, 2618125270, 2580975920,  323434670,  352372614])

In [55]:
df.head(5)

,userId,postId,tagId,score,predicted_score
0,1114791014,7031720382,102871,0,0.031885
1,453980094,5351672082,830883,0,0.023767
2,2738524875,5325077382,3464496,0,0.144696
3,53693143,9836841382,1381366,1,0.044900
4,618968903,5837607382,4474183,0,0.034953


In [57]:
df[df.userId.isin(t.index.values)]

,userId,postId,tagId,score,predicted_score
46573,2618125270,7919181382,1381366,1,0.278692
86354,2618125270,7967645182,74,0,0.128524
88520,352372614,7212202382,1272240,0,0.023171
126024,2618125270,7981172182,9777582,0,0.143801
207918,352372614,1458832382,1381366,0,0.022506
...,...,...,...,...,...
55210329,352372614,1491271182,8291806,0,0.080947
55339373,176951320,7423722282,711045,0,0.826163
55607095,352372614,7217636382,1272240,1,0.078268
55723968,2618125270,3287529082,102871,0,0.182957


In [65]:
temp = df[df.userId == 176951320].sort_values(by = ['predicted_score'])

In [67]:
ndcg_score(temp.score.values, temp.predicted_score.values)

ValueError: Only ('multilabel-indicator', 'continuous-multioutput', 'multiclass-multioutput') formats are supported. Got binary instead

In [68]:
tr_vals = np.asarray([temp.score.values])
pr_vals = np.asarray([temp.predicted_score.values])

In [69]:
ndcg_score(tr_vals,pr_vals)

0.7520911855422138

In [77]:
ndcg_score(np.array([[3,1,1,0,0]]), np.array([[2,1,0.2,0,0]]))

1.0

In [111]:
roc_auc_score(tr_vals[0], pr_vals[0])

0.375

In [112]:
tr_vals

array([[1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]])

In [127]:
relevant_recommendations = np.where(tr_vals[0] == 1)[0]
ordered_recommendations = np.argsort(-pr_vals[0])
apk(relevant_recommendations, ordered_recommendations)

0.33999999999999997

array([ 0,  2,  3,  8, 12])

In [123]:
ordered_recommendations

array([12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0])

In [125]:
tr_vals[0]

1

In [149]:
path = "./train_test_data_models/Malayalam/is_vp_succ2/test_later.txt"

In [150]:
with open(path) as f:
    lines = f.readlines()
    labels = [int(l.replace('\n', '')[0]) for l in lines]

In [151]:
labels[:5]

[0, 0, 0, 0, 0]

In [152]:
predicted_scores[:5]

[0.0318848, 0.0237671, 0.144696, 0.0449003, 0.0349526]

In [153]:
roc_auc_score(labels, predicted_scores)

0.8400228558390365

In [154]:
lines[0]

'0 0:347890:1 1:1475:1 \n'

In [155]:
lines[0].split(':')

['0 0', '347890', '1 1', '1475', '1 \n']

In [186]:
a = {
    "c1": [1,2,2,3,2,2,4,5,5,9],
    "c2": [1,0,0,0,1,1,1,0,1,0]
}

In [187]:
t_df = pd.DataFrame(a)

In [194]:
t_df.groupby("c1").filter(lambda x: x.c1.max() < 5)

,c1,c2
0,1,1
1,2,0
2,2,0
3,3,0
4,2,1
5,2,1
6,4,1


In [202]:
def apply_to_group(x, selected_user_mappings):
    if x.user_mapping in selected_user_mappings:
        true_vals = x.score.values
        predicted_vals = x.predicted_score.values
        print(true_vals, predicted_vals)
        user_ndcg_score = ndcg_score(true_vals[np.newaxis, :], predicted_vals[np.newaxis, :])
        relevant_recommendations = np.where(true_vals == 1)[0]
        ordered_recommendations = np.argsort(-predicted_vals)
        user_apk_score = apk(relevant_recommendations, ordered_recommendations)
        return (user_ndcg_score, user_auc_score, user_apk_score)
    else:
        return None, None, None

In [5]:
def mp_1(p1, p2):
    p1 = p1[0]
    p2 = p2[0]
    return [p1+p1*p2, p1-p1*p2]
params = [([1],[2]),([2],[3]),[[3],[4]],[[4],[5]]]
with Pool(processes=10) as pool:
    res = pool.starmap(mp_1, params)

In [4]:
res

[[3, -1], [8, -4], [15, -9], [24, -16]]

In [6]:
res

[[3, -1], [8, -4], [15, -9], [24, -16]]

In [9]:
l = [np.arange(3), np.arange(3)+2]

In [10]:
l

[array([0, 1, 2]), array([2, 3, 4])]

In [11]:
np.concatenate(l)

array([0, 1, 2, 2, 3, 4])

In [12]:
np.array(l).mean(axis = 0)

array([1., 2., 3.])

In [7]:
a = {
    "c1": [1,2,2,3,2,2,4,5,5,9],
    "c2": [1,0,0,0,1,1,1,0,1,0]
}

In [9]:
df = pd.DataFrame(a)

In [21]:
temp_1 = df.sort_values('c1').c2.values

In [12]:
temp = df.sort_values('c1').c1.values

In [16]:
_, index = np.unique(temp, True)

In [23]:
np.split(temp_1, index[1:])

[array([1]),
 array([0, 0, 1, 1]),
 array([0]),
 array([1]),
 array([0, 1]),
 array([0])]

In [24]:
df.sort_values('c1')

,c1,c2
0,1,1
1,2,0
2,2,0
4,2,1
5,2,1
3,3,0
6,4,1
7,5,0
8,5,1
9,9,0


In [39]:
from sklearn.metrics import ndcg_score
from math import log
# we have groud-truth relevance of some answers to a query:
true_relevance = np.asarray([[1, 1, 0, 0, 1, 0]])
# we predict some scores (relevance) for the answers
scores = np.asarray([[.9, .85, .8, .7, .65, 0.15]])
ndcg_score(true_relevance, scores)

# scores = np.asarray([[.05, 1.1, 1., .5, .0]])
# ndcg_score(true_relevance, scores)

# we can set k to truncate the sum; only top k answers contribute.


# the normalization takes k into account so a perfect answer
# would still get 1.

0.9469024295259745

In [35]:
ndcg_score(true_relevance, scores, k=3)

0.7653606369886218

In [32]:
true_relevance = np.asarray([[1, 1, 0, 0]])
# we predict some scores (relevance) for the answers
scores = np.asarray([[.9, .85, .8, .7]])
ndcg_score(true_relevance, scores)

1.0

In [40]:
1/log(2,2)+1/log(3,2)+1/log(4,2)

2.1309297535714573

In [41]:
1/log(2,2)+1/log(3,2)

1.6309297535714573

In [42]:
1.63/2.13

0.7652582159624413

In [1]:
temp_df = pd.read_csv("./train_test_data_models/Kannada/is_vp_succ2/user_post_ffm_mapping.csv")

NameError: name 'pd' is not defined